## Prerequisites

### mount

**clone the repository**

In [73]:
!git clone https://github.com/vnagireddy-symphonize/resume_parser.git

fatal: destination path 'resume_parser' already exists and is not an empty directory.


**mount google drive (optional)**

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### xterm

In [75]:
!pip install colab-xterm

In [76]:
%load_ext colabxterm
%xterm

The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


Launching Xterm...

<IPython.core.display.Javascript object>

## Installation

In [77]:
!pip install resume-parser

### spacy dependencies

In [78]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz
!pip install importlib-metadata==3.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 22.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### setup spacy and nltk

In [79]:
# spaCy
!python -m spacy download en_core_web_sm

# nltk
!python -m nltk.downloader stopwords
!python -m nltk.downloader punkt
!python -m nltk.downloader averaged_perceptron_tagger
!python -m nltk.downloader universal_tagset
!python -m nltk.downloader wordnet
!python -m nltk.downloader brown
!python -m nltk.downloader maxent_ne_chunker

DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz#egg=en_core_web_sm==2.3.1 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Using cached en_core_web_sm-2.3.1-py3-none-any.whl
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after imp

## Usage

In [80]:
from resume_parser import resumeparse
import json

data = resumeparse.read_file('/content/resume_parser/Kumar\'s Resume.pdf')
print(json.dumps(data, indent=4))


{
    "email": "kumarrajwani1811@gmail.com",
    "phone": "8141873476",
    "name": "Kumar Rajwani",
    "total_exp": 5,
    "university": [],
    "designition": [
        "machine learning engineer",
        "learning engineer",
        "python developer",
        "content writer"
    ],
    "degree": [],
    "skills": [
        "NLP Computer Vision Flask Tensorflow",
        "Pytorch NLTK SKlearn django rasa",
        "Data Preprocessing Keras Web Scraping",
        "Transfer Learning TFLite docker git",
        "Hindi",
        "Professional Working Proficiency",
        "English",
        "Gujarati"
    ],
    "Companies worked at": []
}


## Training

### import dependencies

In [81]:
import spacy
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
import os
import time
import datetime

In [82]:
def ptimeit(func):
  """Decorator to measure execution time of a function."""
  def wrapper(*args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Function '{func.__name__}' executed in {execution_time:.4f} seconds.")
    return result
  return wrapper

### load model

In [83]:
nlp = spacy.load('en_core_web_sm')

**instantiate matcher**

In [84]:
matcher = Matcher(nlp.vocab)

**read titles**

In [85]:
base_path = "/content/resume_parser/resume_parser/"
file = os.path.join(base_path,"titles_combined.txt")
file = open(file, "r", encoding='utf-8')
designation = [line.strip().lower() for line in file]

In [86]:
print(designation)

['.net developer', '.net programmer', '.net architect', '1st grade teacher', '1st pressman', '1st pressman on web press', '2 year olds preschool teacher', '21 dealer', '2nd grade teacher', '2nd pressman', '3d animator', '3d artist', '3d designer', '3d modeler', '3d specialist', '3d technologist', '3rd grade reading teacher', '3rd grade teacher', '3rd pressman', '4-h youth development specialist', '411 directory assistance operator', '4th grade math teacher', '4th grade teacher', '5th grade teacher', '6th grade teacher', '7th grade social studies teacher', '7th grade teacher', '8th grade mathematics teacher', '8th grade teacher', '911 dispatcher', '911 emergency dispatcher', '911 emergency services dispatcher', '911 operator', '911 telecommunicator', 'a auxiliary', 'a class lineman', 'a p supervisor', 'a and p mechanic', 'a and p technician', 'a&p mechanic', 'a&p technician', 'a-operator', 'a/c tech', 'a/c technician', 'aba therapist', 'aba tutor', 'abap developer', 'ac/dc rewinder', 'a

In [87]:
designitionmatcher = PhraseMatcher(nlp.vocab)

**generate spaCy Doc objects**

In [88]:
import time
import datetime

start_time = time.time()
print(f"start_time: {start_time}")

job_title_patterns = [nlp.make_doc(text) for text in designation if len(nlp.make_doc(text)) < 10]

end_time = time.time()
print(f"end_time: {end_time}")
execution_time = end_time - start_time
print(f"Execution time (seconds): {execution_time}")
print(f"Execution time (minutes): {execution_time / 60}")
print(f"Execution Time (h:mm:ss.us): {datetime.timedelta(seconds=execution_time)}")


start_time: 1727824661.493611
end_time: 1727824836.9022791
Execution time (seconds): 175.40866804122925
Execution time (minutes): 2.923477800687154
Execution Time (h:mm:ss.us): 0:02:55.408668


In [89]:
len(job_title_patterns)

74370

In [90]:
import pickle

**save as pre-processed patterns**

In [91]:
pickle.dump(job_title_patterns, open("/content/drive/My Drive/resume-parser/data/processed/kb/job-titles.ppp", "wb"))

In [ ]:
loaded_job_title_patterns = pickle.load(open("/content/drive/My Drive/resume-parser/data/processed/kb/job-titles.ppp", "rb"))
len(loaded_job_title_patterns)

KeyboardInterrupt: 

In [ ]:
job_title_patterns[0], loaded_job_title_patterns[0]

**search the knowledgebase to find the job titles in the resume**

In [ ]:
# prompt: read resume text into resume_as_text using tika

from tika import parser

@ptimeit
def read_resume_text(resume_path):
  """Reads resume text using Tika."""
  parsed = parser.from_file(resume_path)
  return parsed['content']

resume_as_text = read_resume_text('/content/resume_parser/Kumar\'s Resume.pdf')
print(resume_as_text)


In [ ]:
# prompt: convert resume_as_text to lower case

resume_as_text_lower = resume_as_text.lower()


In [ ]:
designitionmatcher.add("JOB_TITLE", None, *loaded_job_title_patterns)

In [ ]:
doc = nlp(resume_as_text.lower())
matches = designitionmatcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)


## Q & A

##### What is the generated pkl file?

In [ ]:
# prompt: what is this generated pkl file?

# The generated "patterns.pkl" file contains a pickled list of spaCy Doc objects.
# These Doc objects represent the different job titles or designations that were
# loaded from the "titles_combined.txt" file.
#
# The purpose of pickling these patterns is to save them to disk for later reuse.
# Instead of having to generate the Doc objects from the text file every time
# the program is run, it can simply load the pre-generated patterns from the
# "patterns.pkl" file using pickle.load.
#
# This saves time and resources as it avoids the need to process the text file
# and generate the spaCy Doc objects every time.

# In essence, the "patterns.pkl" file is a serialized version of a list of
# spaCy Doc objects that are used for pattern matching in the resume parsing process.


##### Can the pkl file be called model?

In [ ]:
# prompt: can this file be called as a model?

# No, this file cannot be strictly called a model in the traditional sense.
#
# While it does contain patterns and data structures that are used for
# processing and extracting information from resumes, it lacks the core
# components of a machine learning model.
#
# Here's why:
# 1. Rule-Based Approach: It primarily uses rule-based methods and pattern
# matching (via spaCy's Matcher and PhraseMatcher) to identify job titles and
# other entities. It does not rely on training a statistical model on a
# large dataset to make predictions or classifications.
# 2. No Learning: There is no training phase where a model learns from a
# dataset to optimize its performance.
#
# However, it can be considered a pre-processing or knowledge base component
# that assists in the resume parsing task. The patterns defined in the
# "patterns.pkl" file are essentially a collection of expertise or rules that
# aid in extracting relevant information.
#
# In essence, this file provides a form of structured knowledge (job titles)
# that can be leveraged for parsing and understanding resumes.
#
# To create a true machine learning model for resume parsing, you would need
# to use techniques like Named Entity Recognition (NER) or text classification.
# This would typically involve training a model on a labeled dataset of resumes
# and extracting entities or predicting attributes (e.g., skills, experience).


##### What are the start and end in thisi context?

In [ ]:
# prompt: What are start and end in this context?

# In the context of the provided code, specifically the loop:
#
# for match_id, start, end in matches:
#
# `start` and `end` represent the starting and ending token indices of the matched phrase within the spaCy Doc object (`doc`).
#
# For example:
#
#  - `start`:  Indicates the index of the first token in the `doc` that corresponds to the beginning of the matched phrase.
#  - `end`:  Indicates the index of the token immediately following the last token in the matched phrase.
#
# Essentially, `start` and `end` define a slice within the `doc` that represents the identified match.

# Let's illustrate with an example:
#
# doc = nlp("Barack Obama urges Congress to find courage to defend his healthcare reforms")
# matches = matcher(doc)
#
#  - If the matcher finds "Barack Obama", `start` would be the index of "Barack" and `end` would be the index of "Obama" + 1 (as it's the token immediately after "Obama").
#
#  - If the matcher finds "healthcare reforms", `start` would be the index of "healthcare" and `end` would be the index of "reforms" + 1.

# The `span = doc[start:end]` line then creates a span object from the Doc that represents the extracted matched phrase.

# In summary, `start` and `end` are crucial variables in the spaCy matching process that help pinpoint the location of matched phrases within a text.


##### What is spaCy KnowledgeBase?

In [ ]:
# prompt: what is spacy knowledgebase?

# A spaCy KnowledgeBase is a component that allows you to store and retrieve
# information about entities, such as their descriptions, aliases, and links
# to external resources. It's often used for tasks like Named Entity Linking
# (NEL), where you want to identify entities in text and connect them to
# entries in a knowledge base.

# It's important to note that the code provided previously doesn't use a
# spaCy KnowledgeBase directly. It uses a custom approach with Matcher and
# PhraseMatcher to extract information from resumes, particularly job titles.

# How a spaCy KnowledgeBase works:
# 1. Entity Linking: It helps link entities found in your text to specific
# entries in your knowledge base.
# 2. Candidate Generation: It can generate a list of potential candidates
# for an entity based on the text and the knowledge base entries.
# 3. Candidate Scoring: It can score these candidates to determine the most
# likely match based on factors like alias similarity, context, and prior
# knowledge.
# 4. Storing and Retrieving Knowledge: It provides a way to organize and
# manage the information in your knowledge base.

# In the context of resume parsing, you could potentially use a spaCy
# KnowledgeBase to store and retrieve information about skills, job titles,
# companies, or universities mentioned in resumes. It would allow you to
# link these entities to detailed entries in your knowledge base and
# extract richer information from the resumes.

# However, the code provided in the previous examples focuses on
# rule-based pattern matching rather than using a spaCy KnowledgeBase.
